In [1]:
import textwrap

# LangChain and AWS Bedrock imports
from langchain.prompts import ChatPromptTemplate
from langchain_aws.chat_models import ChatBedrockConverse
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Initialize LLM
llm = ChatBedrockConverse(model="us.meta.llama3-1-70b-instruct-v1:0", region_name="us-east-1")
print(llm)

disable_streaming='tool_calling' client=<botocore.client.BedrockRuntime object at 0x791d3d32f310> model_id='us.meta.llama3-1-70b-instruct-v1:0' region_name='us-east-1' provider='us' supports_tool_choice_values=()


In [3]:

# INDEXING STEP 1: Load Data
URL = "https://blog.langchain.dev/generating-usable-text-with-ai/"
loader = WebBaseLoader(URL)
blog_data = loader.load()

metadata = blog_data[0].metadata
page_content = blog_data[0].page_content

print(f"Loaded {len(blog_data)} document(s).")
print("Document metadata:", metadata)
print(f"Page content length: {len(page_content)} characters.")
print("Document content (excerpt):")
print("\n".join(textwrap.wrap(page_content, width=80)))

Loaded 1 document(s).
Document metadata: {'source': 'https://blog.langchain.dev/generating-usable-text-with-ai/', 'title': 'Generating Usable Text with AI', 'language': 'en'}
Page content length: 14929 characters.
Document content (excerpt):
   Generating Usable Text with AI
Skip to content                 Case Studies     In the Loop     LangChain
Docs     Changelog      Sign in Subscribe                    Generating Usable
Text with AI  By LangChain 9 min read Feb 5, 2024      Editor's Note: This post
was written by Mutt Data through LangChain's Partner
Program.IntroductionOverviewIn our previous discussions, we not only delved into
the challenges of implementing Generative AI applications in general but also
explored effective mitigation strategies for image generation problems. Now,
it's time to shift our focus to the unique set of challenges that arise when
generating text.In this blog post, we provide a concise overview of these
challenges while sharing some insights from our ex

In [4]:
# INDEXING STEP 2: Splitting
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
splits = text_splitter.split_documents(blog_data)

print(f"\nLoaded data is split into {len(splits)} smaller chunks.")
print(f"Type of each split: {type(splits[0])}")
print("\nContent of one of the chunks (excerpt):")
print("\n".join(textwrap.wrap(splits[10].page_content, width=100)))


Loaded data is split into 17 smaller chunks.
Type of each split: <class 'langchain_core.documents.base.Document'>

Content of one of the chunks (excerpt):
presented the patient with available appointments until one was confirmed.Simplified diagram of
states(light blue) for our medical appointment scheduling application. First managing the learning
of user information and after it matching the right specialty and filtering possible doctors and
time slots from a database4) Evaluating performance Challenge: Measuring performance and getting
unbiased metrics on the quality of text generations. Solution: Selecting a standarized tool for
testing and evaluating LLMsExample: This was a clear issue we faced when implementing our medical
appointment scheduling application. As a first approach we did a test suite of possible prompts
corresponding to a set of topics. Referring to consultations that should get an appointment, others
that were not able to be fulfilled due to the lack of a doctor wi

In [5]:
# INDEXING STEP 3: Embedding
embeddings = HuggingFaceEmbeddings()
print("\nEmbedding the first 40 characters of the 10th split:")
print("Text:", splits[10].page_content[:40])
print("Embedding (first 10 dimensions):", embeddings.embed_query(splits[10].page_content[:40])[:10])

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



Embedding the first 40 characters of the 10th split:
Text: presented the patient with available app
Embedding (first 10 dimensions): [0.023445388302206993, -0.043753959238529205, -0.036080505698919296, -0.05090802535414696, -0.015418066643178463, -0.0016566633712500334, 0.015300992876291275, -0.012591246515512466, 0.021811315789818764, 0.0022387499921023846]


In [6]:
# INDEXING STEP 4: Storing
vectorstore = Chroma.from_documents(documents=splits, embedding=HuggingFaceEmbeddings())

# Retrieval and Generation STEP 1: Retrieval
retriever = vectorstore.as_retriever()

# Retrieval and Generation STEP 2: Querying
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm

# Query example
response = rag_chain.invoke("What are the challenges of generating texts with AI?")
print("\nResponse:")
print("\n".join(textwrap.wrap(response.content, width=80)))


Response:
  The challenges of generating texts with AI include the need for clear and
specific prompts, the difficulty of managing internal states to reduce reliance
on the LLM's memory, and the importance of controlling inputs and outputs to
avoid potential pitfalls. Additionally, the text highlights the need for
strategic considerations, such as task segmentation and robust evaluation
practices, to ensure effective LLM applications.
